In [ ]:
# import numpy as np
# import pandas as pd
# import scipy
# import sklearn
# import spacy
# import matplotlib.pyplot as plt
# import seaborn as sns

# import re
# from collections import Counter

# from time import time
# %matplotlib inline

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import seaborn as sns
import nltk
sns.set_style("whitegrid")
# import altair as alt
# alt.renderers.enable("notebook")

# Code for hiding seaborn warnings
import warnings
warnings.filterwarnings("ignore")
pd.options.display.max_rows = 4000

In [2]:
# df_path = "/home/lnc/0. Latest News Classifier/01. Dataset Creation/"
df_path = 'data/'
df_temp1 = pd.read_csv(df_path + 'articles1.csv')
df_temp2 = pd.read_csv(df_path + 'articles2.csv')
df_temp3 = pd.read_csv(df_path + 'articles3.csv')

df_temp = df_temp1.append(df_temp2).append(df_temp3)

In [3]:
df = df_temp.set_index('id')[['title','publication', 'year', 'content']]
df['content_len'] = df['content'].apply(len)

In [4]:
df.head()

,title,publication,year,content,content_len
id,,,,,
17283,House Republicans Fret About Winning Their Hea...,New York Times,2016.0,WASHINGTON — Congressional Republicans have...,5607
17284,Rift Between Officers and Residents as Killing...,New York Times,2017.0,"After the bullet shells get counted, the blood...",27834
17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,2017.0,"When Walt Disney’s “Bambi” opened in 1942, cri...",14018
17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,2017.0,"Death may be the great equalizer, but it isn’t...",12274
17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,2017.0,"SEOUL, South Korea — North Korea’s leader, ...",4195


In [5]:
# len(df.sort_values(by = 'content_len').iloc[200].content)

In [7]:
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))

LookupError: 
**********************************************************************
  Resource 'corpora/stopwords' not found.  Please use the NLTK
  Downloader to obtain the resource:  >>> nltk.download()
  Searched in:
    - 'C:\\Users\\test/nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:\\Users\\test\\Anaconda3\\nltk_data'
    - 'C:\\Users\\test\\Anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\test\\AppData\\Roaming\\nltk_data'
**********************************************************************

In [6]:
df.shape

(142570, 5)

In [ ]:
nltk.download()

In [ ]:
def unlist(pd_col):
    flat_list = ''
    for sublist in pd_col:
        for item in sublist:
            flat_list + item
    return flat_list

In [ ]:
df_2 = df.loc[df.content_len.between(10000, 150000)]

In [ ]:
df_2.head()

In [ ]:
df_2['full_content'] = df_2['title'] + ' ' + df_2['content']

In [ ]:
df_3 = df_2[['publication', 'full_content', 'content_len']]

In [ ]:
'dsfsdf'.lower()

In [ ]:
df_3['full_content'] = df_3['full_content'].str.lower()

In [ ]:
nltk.download()

In [ ]:
word_tokens = nltk.word_tokenize(df_3['full_content'])

In [ ]:
df_2['content_flat'] = map(unlist, df_2['content'])

In [ ]:
df_2['content'].iloc[20]

In [ ]:
df_2.head()

In [ ]:
flat_list = [item for sublist in l for item in sublist]

In [ ]:
import spacy

In [ ]:
nlp 

In [ ]:
!pip install -U spacy

In [ ]:
!pip install spacy spacy download en_core_web_md

In [ ]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_md-2.0.0/en_core_web_md-2.0.0.tar.gz

In [ ]:
!python -m spacy link ./spacymodels/en_core_web_md-2.0.0/en_core_web_md en_core

In [ ]:
nlp = spacy.load('en_core_web_sm', parse=True, tag=True, entity=True)

In [ ]:

# Tools for processing data
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, classification_report, confusion_matrix, make_scorer, adjusted_rand_score, silhouette_score, homogeneity_score, normalized_mutual_info_score
# Classifiers, supervised and unsupervised
from sklearn import ensemble
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AffinityPropagation

In [ ]:
#nlp_vec = spacy.load('en_vecs', parse = True, tag=True, #entity=True)
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

In [ ]:
((df.sort_values(by = 'content_len').loc[df.content_len ==10001, 'content']))

In [ ]:
sns.distplot(df.loc[df.content_len.between(10000, 150000),'content_len'], bins=20)
plt.show()

In [ ]:
min_len = 100
for i in range(len(df.content)):
    if len(df.content.iloc[i]):
        min


In [ ]:
len(df.content.iloc[]

In [ ]:
for i in df.content:
    print(i)

In [ ]:
import spacy

In [ ]:
# Load spacy NLP object
nlp = spacy.load('en_core_web_sm')

In [ ]:
# t0 = time()



# A list to store common words by all authors
common_words = []

# A dictionary to store the spacy_doc object of each author
authors_docs = {}

# for name in names:
    # Corpus is all the text written by that author
corpus = ""
    # Grab all rows of current author, along the 'content' column
#     author_content = authors_data.loc[authors_data.author==name,'content']
    

In [ ]:
df = df.loc[df.content_len.between(20000,40000)]

In [ ]:
df.shape

In [ ]:

# Merge all articles in to the author's corpus
for article in df['content']:
    corpus = corpus + article
# Let Spacy parse the author's body of text
doc = nlp(corpus)

# Store the doc in the dictionary
authors_docs[name] = doc

# Filter out punctuation and stop words.
lemmas = [token.lemma_ for token in doc
            if not token.is_punct and not token.is_stop]

# Return the most common words of that author's corpus.
bow = [item[0] for item in Counter(lemmas).most_common(1000)]

# Add them to the list of words by all authors.
for word in bow:
    common_words.append(word)

# Eliminate duplicates
common_words = set(common_words)
    
print('Total number of common words:',len(common_words))
print("done in %0.3fs" % (time() - t0))

In [ ]:
cat = df['content']

In [ ]:
cat

In [ ]:
cat.to_list()[1]

In [ ]:
cat.iloc[2]

In [ ]:
cat.to_list()[2]

In [ ]:
# figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')

In [ ]:
sns.set(style="darkgrid")
fig= plt.figure(figsize=(20,20))
sns.countplot(x="publication", data=df)
plt.show()

In [ ]:
fox, breitbart, vox, npr, buzzword, new york times, guardian,

In [ ]:
df.head()

In [ ]:
df.group

In [ ]:
df.shape

In [ ]:
len(df.id.unique())

In [ ]:
lengths = pd.Series([len(x) for x in data.content])
print('Statistical Summary of Article Lengths')
print(lengths.describe())

sns.distplot(lengths,kde=False)
plt.title('Distribution of Article Lengths (All)')
plt.show()
sns.distplot(lengths[lengths<10000],kde=False)
plt.title('Distribution of Articles Lengths < 10,000 Characters')
plt.show()